In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
corner_params = dict(maxCorners = 10,qualityLevel = 0.3,minDistance = 7,blockSize = 7)

In [3]:
lk_params = dict(winSize = (200,200),maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))

In [11]:
cap = cv2.VideoCapture(0)
ret,prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)
old_pts = cv2.goodFeaturesToTrack(prev_gray,mask = None, **corner_params)
mask = np.zeros_like(prev_frame)

while True:
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    next_pts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,old_pts,None, **lk_params)
    
    good_new = next_pts[status == 1]
    good_old  = old_pts[status == 1]
    
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        x_new,y_new = new.ravel()
        x_old,y_old = old.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_old,y_old),(0,255,0),3)
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('Object Tracking',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    prev_gray = frame_gray.copy()
    old_pts = next_pts.reshape(-1,1,2)


cv2.destroyAllWindows()
cap.release()
